In [1]:
import time
from datetime import datetime

def format_data_for_modelling(schema, training_data, label_data):
    """
    Takes in a list of headers, a training data set and info on the eventual
    outcome (a.k.a label/dependent variable/flag), merges and reformats the
    data to generate a list of feature vectors on which a logistic regression
    model may be built. All features values must be real numbers.

    :param schema: list of json representing column types, names and other info.
        Every entry will have at least a 'type' and 'name' key and potentially
        others. Types will be one of {id, date, datetime, str, float, int,
        bool}.  Names will be unique across entries.
            e.g. [{'type': 'int', 'name': 'my_field'}, ...]

    :param training_data: list of lists where each internal list represents a
        record to model. Columns follow types from schema.

    :param label_data: dict of {id: bool} representing the dependent variable
        (label) of the data set.

    :return: list of lists of all numbers representing features capable of
        having logistic regression run on them. The first col should have the
        id and the final col should have the dependent variable/label flag
            e.g. [[1,17,3,95.0,...,1],[2,34,5,96.0,...,0],...]
    """
    ALLOWED_TYPES = {'id', 'date', 'datetime', 'str', 'float', 'int', 'bool'}

    # ensure all column definitions contain at least type and name
    assert all(['type' in x and 'name' in x for x in schema])
    # ensure there is one and only one column with type id
    assert sum([x['type'] == 'id' for x in schema]) == 1
    # ensure all rows in training match len of column definitions
    assert all(len(x) == len(schema) for x in training_data)
    # ensure all types are standard
    assert all([col['type'] in ALLOWED_TYPES for col in schema])
    # ensure all names are unique
    col_names = [col['name'] for col in schema]
    assert all([col_names.count(col['name']) == 1 for col in schema])
    
    #implement your code here and output a 2D array:
    
    # categories = dict()
    
    # unique vals for str field
    strs = set([row[2] for row in training_data])
    counts = len(strs)
    uniques = {ss:idx for idx, ss in enumerate(strs, 0)}
    
    
    all_data = []
    for row in training_data:
        translated = []
        # str_trans = [0] * counts  {'a':0, 'b':1, 'c':2 ...}
        for idx, sc in enumerate(schema[1:], 1):
            if sc['type'] == 'date':
                translated.append(time.mktime(datetime.strptime(row[idx], sc['date_format']).timetuple()))
            elif sc['type'] == 'str':
                # cat = categories.get(row[idx], 0)
                # if cat == 0:
                #     categories[row[idx]] = len(categories) + 1
                # translated.append(categories[row[idx]])
                rr = [0] * counts
                rr[uniques[row[idx]]] = 1
                translated.extend(rr)
            elif sc['type'] in ['float', 'int']:
                translated.append(row[idx])
            elif sc['type'] == 'bool':
                translated.append(int(row[idx]))
            elif sc['type'] == 'datetime':
                translated.append(time.mktime(datetime.strptime(row[idx], sc['date_format']).timetuple()))
                
        all_data.append(translated)        
    
    return all_data   
    # return [
    #     # [123, <date features>, <categorical features>, ..., 1],
    #     # [123, <..>, <..>, ..., 0],
    # ]

#-----------------Example Input-------------------#

# it's ok to modify this schema to supply any extra info required
example_schema = [
    {'type': 'id', 'name': 'user_id'},
    {'type': 'date', 'name': 'loan_application_date',
     'date_format': '%m/%d/%Y'},
    {'type': 'str', 'name': 'category_one'},
    {'type': 'float', 'name': 'numerical_field_one'},
    {'type': 'int', 'name': 'numerical_field_two'},
    {'type': 'bool', 'name': 'bool_field_one'},
    {'type': 'datetime', 'name': 'previous_event_date_1',
     'date_format': '%m/%d/%Y %H:%M'},
]

example_training_data = [
    [123, "12/1/2015", "a", 1000.0, 13, True, "2/1/2001 13:45"],
    [124, "6/11/2016", "b", 3500.0, 8, False, "4/13/2010 22:34"],
    [125, "10/12/2017", "c", 1200.0, 0, False, "7/1/2012 15:08"],
    [126, "11/10/2015", "a", 11000.0, 153, True, "12/10/1994 02:14"],
    [127, "7/5/2016", "c", 150.0, 55, True, "9/3/2004 19:55"],
]

example_label_data = {
    123: True,
    124: False,
    126: False,
    127: True
}

for line in format_data_for_modelling(
    example_schema,
    example_training_data,
    example_label_data
):
    print(line)

[1448946000.0, 1, 0, 0, 1000.0, 13, 1, 981053100.0]
[1465617600.0, 0, 1, 0, 3500.0, 8, 0, 1271212440.0]
[1507780800.0, 0, 0, 1, 1200.0, 0, 0, 1341169680.0]
[1447131600.0, 1, 0, 0, 11000.0, 153, 1, 787043640.0]
[1467691200.0, 0, 0, 1, 150.0, 55, 1, 1094255700.0]


In [2]:
 sorted(['aa', 'axax'], key=len)


['aa', 'axax']

In [10]:
# tuple(map(chr, range(ord('a'), ord('z')+1)))

In [18]:
# password generator
import random

LOWERCASE_CHARS = tuple(map(chr, range(ord('a'), ord('z')+1)))
UPPERCASE_CHARS = tuple(map(chr, range(ord('A'), ord('Z')+1)))
DIGITS = tuple(map(str, range(0, 10)))
SPECIALS = ('!', '@', '#', '$', '%', '^', '&', '*')

SEQUENCE = (LOWERCASE_CHARS,
            UPPERCASE_CHARS,
            DIGITS,
            SPECIALS,
            )

def generate_random_password(total, sequences):
    r = _generate_random_number_for_each_sequence(total, len(sequences))
#     print(r)
    password = []
    for (population, k) in zip(sequences, r):
        n = 0
        while n < k:
            position = random.randint(0, len(population)-1)
            password += population[position]
            n += 1
    random.shuffle(password)
    
    while _is_repeating(password):
        random.shuffle(password)
        
    return ''.join(password)

def _generate_random_number_for_each_sequence(total, sequence_number):
    """ Generate random sequence with numbers (greater than 0).
        The number of items equals to 'sequence_number' and
        the total number of items equals to 'total'
    """
    current_total = 0
    r = []
    for n in range(sequence_number-1, 0, -1):
        current = random.randint(1, total - current_total - n)
        current_total += current
        r.append(current)
    r.append(total - sum(r))
    random.shuffle(r)

    return r

def _is_repeating(password):
    """ Check if there is any 2 characters repeating consecutively """
    n = 1
    while n < len(password):
        if password[n] == password[n-1]:
            return True
        n += 1
    return False

if __name__ == '__main__':
    print(generate_random_password(random.randint(6, 30), SEQUENCE))

0*97Y7c@76*%05Q96


In [17]:
_is_repeating('aa')

True

# Following examples are from
http://pythonfiddle.com/knockout-sequence-sorting/

In [20]:
class DFA:
    current_state = None;
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states;
        self.alphabet = alphabet;
        self.transition_function = transition_function;
        self.start_state = start_state;
        self.accept_states = accept_states;
        self.current_state = start_state;
        return;
    
    def transition_to_state_with_input(self, input_value):
        if ((self.current_state, input_value) not in self.transition_function.keys()):
            self.current_state = None;
            return;
        self.current_state = self.transition_function[(self.current_state, input_value)];
        return;
    
    def in_accept_state(self):
        return self.current_state in accept_states;
    
    def go_to_initial_state(self):
        self.current_state = self.start_state;
        return;
    
    def run_with_input_list(self, input_list):
        self.go_to_initial_state();
        for inp in input_list:
            self.transition_to_state_with_input(inp);
            continue;
        return self.in_accept_state();
    pass;



states = {0, 1, 2, 3};
alphabet = {'a', 'b', 'c', 'd'};

tf = dict();
tf[(0, 'a')] = 1;
tf[(0, 'b')] = 2;
tf[(0, 'c')] = 3;
tf[(0, 'd')] = 0;
tf[(1, 'a')] = 1;
tf[(1, 'b')] = 2;
tf[(1, 'c')] = 3;
tf[(1, 'd')] = 0;
tf[(2, 'a')] = 1;
tf[(2, 'b')] = 2;
tf[(2, 'c')] = 3;
tf[(2, 'd')] = 0;
tf[(3, 'a')] = 1;
tf[(3, 'b')] = 2;
tf[(3, 'c')] = 3;
tf[(3, 'd')] = 0;
start_state = 0;
accept_states = {2, 3};

d = DFA(states, alphabet, tf, start_state, accept_states);

inp_program = list('abcdabcdabcd');

print(d.run_with_input_list(inp_program))


False


In [23]:
import math

i=12.0
q=5.0

print("Cartesian: (%.1f,%.1f)" %(i,q))
print("Polar: r = %.1f" %math.sqrt(math.pow(i,2) + math.pow(q,2)) + ", Theta = %.1f" %math.degrees(math.atan(q/i)) + " degrees")



Cartesian: (12.0,5.0)
Polar: r = 13.0, Theta = 22.6 degrees


## This script parses GPX stream and convert to LatLong pairs. The script is based on Pi Marillion's answer: http://stackoverflow.com/a/19470243/2327005

In [25]:
#! /usr/bin/env python

# use the regex module
import re

# read in the file
# in_file = open('source').read()

# or use string as input
in_file = '''
	 <trk>
        <trkseg>
            <trkpt lat="38.8577288" lon="-9.0997973"/>
            <trkpt lat="38.8576367" lon="-9.1000557"/>
            <trkpt lat="38.8575259" lon="-9.1006374"/>
        </trkseg>
    </trk>
'''

# Find matches using regex
matches = re.findall('<trkpt lat="([-0-9\.]+)" lon="([-0-9\.]+)"/>', in_file)

# make new file lines by combining lat and lon from matches
out_lines = [lon + ',' + lat for lat, lon in matches]

# convert array of strings to single string
out_lines = '\n'.join(out_lines)

print(out_lines)
# output to new file
# open('dest', 'w').write(out_lines)

-9.0997973,38.8577288
-9.1000557,38.8576367
-9.1006374,38.8575259


In [29]:
def greatest_common_divisor(x,y):
    print("For", x, "and", y,","  )
    r=x%y
    while r>0:
        r=x%y
        if r ==0: 
            print("the greatest common divisor is", y,".")
        else:
            q=y
            x=q
            y=r

greatest_common_divisor(1071,1029)
greatest_common_divisor(5538,1105)
greatest_common_divisor(8764,7732)        



For 1071 and 1029 ,
the greatest common divisor is 21 .
For 5538 and 1105 ,
the greatest common divisor is 13 .
For 8764 and 7732 ,
the greatest common divisor is 4 .


In [32]:
# Add a max operator which will return the max value held in the stack in constant time

import random
# max operation on a stack

class Node:
  def __init__(self):
    self.data = None # contains the data

class StackNode:
  def __init__(self):
    self.maxNode = None # contains the data  
    self.nextNode = None

class Stack:
  def __init__(self):
    self.head = None      
        
  def push(self, node):
     
    toAdd = StackNode()       
    if self.head:
      toAdd.nextNode = self.head
      if node.data > self.head.maxNode.data:
        toAdd.maxNode = node
      else:
        toAdd.maxNode = self.head.maxNode               
    else:
      toAdd.maxNode = node
    self.head = toAdd  
        
       
  def pop(self):
    toReturn = None  
    if self.head:
      toReturn = self.head
      if self.head.nextNode:
        self.head = self.head.nextNode
      else:
        self.head = None
        
    return toReturn  
      
  def max(self):
   return self.head.maxNode.data 
    
    
    
stack = Stack()
for i in range (0,10):
  node = Node()
  node.data = random.randint(1,20)
  print("Pushing: " + str(node.data))
  stack.push(node)

print(stack.max()  )

Pushing: 9
Pushing: 3
Pushing: 2
Pushing: 9
Pushing: 15
Pushing: 4
Pushing: 5
Pushing: 10
Pushing: 11
Pushing: 10
15


In [33]:
stack

In [35]:
# Reorder a linked list such that 
# n0 -> n1 -> n2 -> n3 -> n4 -> n5

# becomes 

# n0 -> n2 -> n4 -> n1 -> n3 -> n5


class Node:
  def __init__(self):
    self.data = None # contains the data
    self.nextNode = None # contains the reference to the next node

# create a linked list without a cycle    
head = Node()
head.data = 0
current = head

for i in range(1,9):
  new_node = Node()
  new_node.data = i
  current.nextNode = new_node
  current = new_node  


def printList(head):
  current = head
  while current.nextNode:
    print(current.data)
    current = current.nextNode

  print(current.data)

def oddsEvens(head):

  temp = None  
  current = head
  head2 = head.nextNode
  count = 1  
  while current.nextNode:
    count += 1    
    temp = current.nextNode
    if temp.nextNode:
      current.nextNode = temp.nextNode
      current = temp
    elif (count % 2 == 0): 
      current.nextNode = head2
      break        
    else:
      current.nextNode = None
      temp.nextNode = head2
      break
        
  printList(head)
  
    
printList(head)    
oddsEvens(head)   
    
  

0
1
2
3
4
5
6
7
8
0
2
4
6
8
1
3
5
7


In [38]:
# Flatten a list using a single line function.

import itertools

flatten = lambda x: list(itertools.chain.from_iterable(x))

s = [['"', 'An', 'investment'], ['in'], ['knowledge'], ['pays'], ['the', 'best'], ['interest."', '--'], ['Benjamin'], ['Franklin']]
print(' '.join(flatten(s)))



" An investment in knowledge pays the best interest." -- Benjamin Franklin


In [40]:
import datetime

my_birthdate = datetime.date(1979,3,12)
my_birthtime = datetime.time(6,0)
my_birthday = datetime.datetime.combine(my_birthdate, my_birthtime)
now = datetime.datetime.now()

time_passed = now - my_birthday
how_many_seconds = time_passed.total_seconds()
print(how_many_seconds)



1294477754.31067


In [41]:
my_birthdate

datetime.date(1979, 3, 12)

In [42]:
my_birthtime

datetime.time(6, 0)

In [43]:
my_birthday

datetime.datetime(1979, 3, 12, 6, 0)

In [46]:
'''This is my first solution on Coderbyte'''

def FirstReverse(str): 

  # code goes here
  ret_str = ''
  for l in range(len(str)-1,-1,-1):
    ret_str += str[l]
  return ret_str
    
    
# keep this function call here  
# to see how to enter arguments in Python scroll down
print(FirstReverse(input()))



test
tset


In [49]:
# Retrieve the column index from a spreadsheet style column alpha ID  
def id2num(s):
    """ spreadsheet column name to number
    http://stackoverflow.com/questions/7261936

   :param s: str -- spreadsheet column alpha ID (i.e. A, B, ... AA, AB,...)
   :returns: int -- spreadsheet column number (zero-based index)

    >>> id2num('A')
    0
    >>> id2num('B')
    1
    >>> id2num('XFD')
    16383
    >>>

    """
    n = 0
    for ch in s.upper():
        n = n * 26 + (ord(ch) - 65) + 1
    return n - 1

print(id2num('A'))
print(id2num('B'))
print(id2num('AA'))
print(id2num('AB'))
print(id2num('XFD'))
print(id2num('BA'))


0
1
26
27
16383
52


In [58]:
# Project Euler Problem # 16
# Takes a number and sums its digits

def PowerDigitSum(value):
    sumDigit = 0
    while value > 0:
        mod = value % 10
        value = value // 10
        sumDigit = sumDigit + mod
    return sumDigit
        
print(PowerDigitSum(2**1000))
print(PowerDigitSum(78))

1366
15


In [63]:
# Definitions
# One Time Pad encryption is a secure 

message = bytearray.fromhex('6c73d5240a948c86981bc294814d')
clear = bytearray('attack at dawn', 'utf-8')
new = bytearray('attack at dusk', 'utf-8')

def mod(a, b):
    c = bytearray()
    for i, j in zip(a, b):
        i, j = bin(i)[2:], bin(j)[2:]
        while len(i) < 8:
            i = '0' + i
        while len(j) < 8:
            j = '0' + j
        k = '0b' + ''.join(str((int(m) + int(n)) % 2) for m, n in zip(i, j))
        k = int(k, 2)
        c.append(k)
    return c

def hexstring(a):
    b = ''
    for i in a:
        i = hex(i)[2:]
        while len(i) < 2:
            i = '0' + i
        b = b + i
    return b

# Execution

if __name__ == '__main__':
    diff = mod(clear, new)
    modif = mod(message, diff)
    if mod(diff, modif) == message:
        print('Trafficking achieved.')
        print(hexstring(modif))

Trafficking achieved.
6c73d5240a948c86981bc2808548


In [62]:
message
bytearray.fromhex('6c73d5240a948c86981bc294814d')

bytearray(b'ls\xd5$\n\x94\x8c\x86\x98\x1b\xc2\x94\x81M')

In [61]:
new

bytearray(b'attack at dusk')

In [67]:
#!/usr/bin/python -tt
# Copyright 2010 Google Inc.
# Licensed under the Apache License, Version 2.0
# http://www.apache.org/licenses/LICENSE-2.0

# Google's Python Class
# http://code.google.com/edu/languages/google-python-class/
# Lecture List
def main():
  general()
  meths()
  
def general():
  colors = ['red', 'blue', 'green']
  print(colors[0])    ## red
  print(colors[2] )   ## green
  print( len(colors) )  ## 3
  
  z = [1,2,3]
  y = [4,5,6]
  x = z + y
  print( z)
  x = (y)
  x[0] = 7
  print( y[0])
  
  squares = [1, 4, 9, 16]
  sum = 0
  for num in squares: sum += num
  print( sum)
  
  list = ['larry', 'curly', 'moe']
  if 'curly' in list: print ('yay')
  
  ## print the numbers from 0 through 99
  for i in range(100): print( i)
    
  ## Access every 3rd element in a list
  a = [1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9,0]
  i = 0
  while i < len(a):
    print (a[i])
    i = i + 3
    
def meths():
  list = ['larry', 'curly', 'moe']
  list.append('shemp')         ## append elem at end
  list.insert(0, 'xxx')        ## insert elem at index 0
  list.extend(['yyy', 'zzz'])  ## add list of elems at end
  print( list ) ## ['xxx', 'larry', 'curly', 'moe', 'shemp', 'yyy', 'zzz']
  print (list.index('curly'))    ## 2

  list.remove('curly')         ## search and remove that element
  print( list.pop(1) )           ## removes and returns 'larry'
  print( list)  ## ['xxx', 'moe', 'shemp', 'yyy', 'zzz']

  list = ['a', 'b', 'c', 'd']
  print( list[1:-1])   ## ['b', 'c']
  list[0:2] = 'z'    ## replace ['a', 'b'] with ['z']
  print (list )        ## ['z', 'c', 'd']
    
# Standard boilerplate to call the main() function.
if __name__ == '__main__':
  main()

red
green
3
[1, 2, 3]
7
30
yay
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
1
4
7
0
3
6
9
['xxx', 'larry', 'curly', 'moe', 'shemp', 'yyy', 'zzz']
2
larry
['xxx', 'moe', 'shemp', 'yyy', 'zzz']
['b', 'c']
['z', 'c', 'd']


In [72]:
'''This will be a Python script that functions as a grocery calculator. It will take in key-value pairs for items
and their prices, and return the subtotal and total, and can print out the list for you for when you're ready to
take it to the store!'''

'''Algorithm:
1. User enters key-value pairs that are added into a dict.
2. Users tells script to return total, subtotal, and key-value pairs in a nicely formatted list.'''

#Object = GroceryList
#Methods = addToList, Total, Subtotal, returnList

class GroceryList(dict):

    def __init__(self):
        self = {}  # here it is directly assigned to the self itself not like self.data!

    def addToList(self, item, price):
            self.update({item:price})

    def Total(self):
        total = 0
        for items in self:
            total += (self[items])*.07 + (self[items])
        return total

    def Subtotal(self):
        subtotal = 0
        for items in self:
            subtotal += self[items]
        return subtotal

    def returnList(self):
        return self

'''Test list should return:
Total = 10.70
Subtotal = 10
returnList = {"milk":4, "eggs":3, "kombucha":3}
'''
List1 = GroceryList()

List1.addToList("milk",4)
List1.addToList("eggs", 3)
List1.addToList("kombucha", 3)


print(List1.Total())
print(List1.Subtotal())
print(List1.returnList())

#*****************************************************
print()
#*****************************************************


List2 = GroceryList()

List2.addToList('cheese', 7.49)
List2.addToList('wine', 25.36)
List2.addToList('steak', 17.64)

print(List2.Total())
print(List2.Subtotal())
print(List2.returnList())

10.7
10
{'milk': 4, 'eggs': 3, 'kombucha': 3}

54.024300000000004
50.49
{'cheese': 7.49, 'wine': 25.36, 'steak': 17.64}


In [74]:
class Tree:
	def __init__(self):
		self.root = None
	def setRoot(self, node):
	    self.root = node        
    	def size(self):
		if self.root == None:
			return 0
		def subtreeSize(node):
			return 1 + sum(subtreeSize(c) for c in node.getChildren())
		return subtreeSize(self.root)
    
class BinaryNode:
	def __init__(self, v):
		self.val = v
		self.leftChild = None
		self.rightChild = None
	def get(self):
		return self.val
	def set(self, v):
		self.val = v
	def getChildren(self):
		children = []
		if self.leftChild != None:
			children.append(self.leftChild)
		if self.rightChild != None:
			children.append(self.rightChild)
		return children
    
class BinarySearchTree(Tree):
    def insert(self, val):
        if self.root == None:
          	self.setRoot(BinaryNode(val))
        else:
            currentRoot = self.root
            while(True):
                if val <= currentRoot.get():
                    if currentRoot.leftChild != None:
                        currentRoot = currentRoot.leftChild
                    else:
                     	currentRoot.leftChild = BinaryNode(val)
                        break
                else:
                    if currentRoot.rightChild != None:
                       currentRoot = currentRoot.rightChild 
                    else:
                        currentRoot.rightChild = BinaryNode(val)
                        break

    def find(self, val):
        currentRoot = self.root
        while (currentRoot != None and currentRoot.get() != val):
            if val < currentRoot.get():
                currentRoot = currentRoot.leftChild
            else:
                currentRoot = currentRoot.rightChild
                
       	if currentRoot == None:
            return False
        else:
            return True

if __name__ == "__main__":
    btree = BinarySearchTree()
    vals = [5, 3, 9, 4, 1, 7, 30]
    for val in vals:
        btree.insert(val)
    print "Tree size is %i" % btree.size()
    tests = [8, 7, 1, 5]
    for t in tests:
        print("find(%i) = %s" % (t, ("True" if btree.find(t) else "False")) )
            

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-74-8f1b6dfc615c>, line 6)

In [98]:
def fib():
    a, b = 0, 1
    while 1:
        yield a
        a, b = b, a+b

f = fib()
# for _ in range(10):
#     print(f.next(), end=',')
    
for _ in range(10):
    print(f.__next__(), end=',')    

0,1,1,2,3,5,8,13,21,34,

In [96]:
dir(f)


['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [99]:
# Find the leftest element in an ordered array using binary search
def findleft(s, left, right, t):
    l = left - 1 # s[l] < t
    r = right + 1 # s[r] >= t
    p = -1
    
    while l+1 != r:
        #invariant:s[l]<t and s[r]>=t
        m = l + (r - l)/2
        if s[m] >= t: # =
            r = m
        else:
            l = m
            
    if r < right and s[r] == t:
        p = r
    return p   
        
s = [7,7,7,9,12,12,12,27,38,38]

print findleft(s, 0, 9, 6)
print 'ok'

SyntaxError: invalid syntax (<ipython-input-99-44dd407d1e44>, line 21)

In [100]:
# Module for generating random numbers in a dice-like fashion
# Authors: Joey Sabey, GameFreak7744@gmail.com
# Initial code in D, adapted by Thomas Lecarpentier
# Copyright: © Joey Sabey 2012
# License: MIT License


import random

'''
  Represents a single n-sided dice
  Example:
  ---
  Dice d8 = new Dice(8);
  d8.roll();
  writeln(d8.result); //prints a random number between 1 and 8
  writeln(d8.result); //prints the same number
  writeln(d8.roll()); //prints a new random number between 1 and 8
 
'''
class Dice :
    	
	def __init__(self,sides = 6) :    
		# by default dice have 6 sides
		self._sides = self.checkSides(sides)
		self._result = 0
    
    	
	'''
	Returns: Current number of sides of the dice
	'''
	def getSides(self):
		return self._sides
        
            
	'''
	Allows you to set the the number of sides of the dice, to anything from 2 to ubyte.max
	'''
	def setSides(self,sides):
		self._sides = self.checkSides(sides)
		return self._sides

	'''
    Returns: Result of the last roll of the dice
    '''
	def getResult(self):
		return self.checkResult()
        
        

	'''
    Returns: A random number from 1 to sides
    '''
	def roll(self):
		self._result = random.randint(1, self._sides)
		return self._result
        
        
	def checkSides(self,sides):
		if sides < 2 :
			raise Error('Dice must have at least 2 sides, passed ' + sides )
		return sides
        
        
	def checkResult():
		if self._result == 0:
			raise Error("Dice has not been rolled yet; must call roll() first")
		return self._result
        
       
if __name__ == "__main__":
        
	dice = Dice()
	print dice.roll()

SyntaxError: invalid syntax (<ipython-input-100-dc704683bb98>, line 74)

In [102]:
def parse_url_string(url):
    params = url.split('?')[1].split('&')
    car = {}
    for param in params:
        car[param.split('=')[0]] = param.split('=')[1]
    return car
    
model_x = parse_url_string('domain.com?model=x&color=black&package=signature&interior=brown&wheels=alloy')

print(model_x)
print('Model: %s' % model_x['model'])
print('Color: %s' %model_x['color'])
print('Package: %s' %model_x['package'])
print('Interior: %s' %model_x['interior'])
print('Wheels: %s' %model_x['wheels'])



{'model': 'x', 'color': 'black', 'package': 'signature', 'interior': 'brown', 'wheels': 'alloy'}
Model: x
Color: black
Package: signature
Interior: brown
Wheels: alloy


In [103]:
# Clock Angle and Max Sum Subarray for Dan Young at Microsoft.
import random

def max_sum_subarray(arr):
  max_found = None
  current_range_sum = 0
  for val in arr:
    current_range_sum += val
    max_found = max(max_found, current_range_sum)
    
    # if a current_range has gotten to be negative,
    # we reset the current_range_sum to 0 because
    # the next contiguous subarray cannot possibly
    # be at its max value if it includes the current,
    # negative subrange.
    current_range_sum = max(current_range_sum, 0)
  return max_found

print "Max sum of a subarray in %s is %d" % ([1,2,3,-5,4], max_sum_subarray([1,2,3,-5,4]))
print "Max sum of a subarray in %s is %d" % ([-8, -23, -14], max_sum_subarray([-8, -23, -14]))
print "Five Random Test Cases"
for x in xrange(5):
  a = [int(10*random.random()) for i in xrange(10)]
  print "Max sum of a subarray in %s is %d" % (a, max_sum_subarray(a))


def clock_angle(h, m):
  h_deg = (h * 30) % 360
  m_deg = (m * 6) % 360
  h_deg += m / 2        # hr moves 30 degrees per 60 minutes
  angle = abs(h_deg - m_deg)
  if angle > 180:
    angle = 360 - angle
  return angle


print "The smallest angle at %d:%02d is %d degrees" % (12, 0, clock_angle(12, 0))
print "The smallest angle at %d:%02d is %d degrees" % (3, 15, clock_angle(3, 15))
print "The smallest angle at %d:%02d is %d degrees" % (6, 0, clock_angle(6, 0))
print "Five Random Test Cases"
for x in xrange(5):
    h, m = int(12*random.random())+1, int(60*random.random())
    print "The smallest angle at %d:%02d is %d degrees" % (h, m, clock_angle(h, m))

SyntaxError: invalid syntax (<ipython-input-103-ddb81c2903b7>, line 19)

In [104]:
#!/usr/bin/python
#
# Stolen from Ero Carrera
# http://blog.dkbza.org/2007/05/scanning-data-for-entropy-anomalies.html

import math, string, sys, fileinput

def range_bytes (): return range(256)
def range_printable(): return (ord(c) for c in string.printable)
def H(data, iterator=range_bytes):
    if not data:
        return 0
    entropy = 0
    for x in iterator():
        p_x = float(data.count(chr(x)))/len(data)
        if p_x > 0:
            entropy += - p_x*math.log(p_x, 2)
    return entropy

def main ():
    for row in fileinput.input():
        string = row.rstrip('\n')
        print ("%s: %f" % (string, H(string, range_printable)))

for str in ['gargleblaster', 'tripleee', 'magnus', 'lkjasdlk',
               'aaaaaaaa', 'sadfasdfasdf', '7&wS/p(']:
    print ("%s: %f" % (str, H(str, range_printable)))



gargleblaster: 2.931209
tripleee: 2.405639
magnus: 2.584963
lkjasdlk: 2.500000
aaaaaaaa: 0.000000
sadfasdfasdf: 2.000000
7&wS/p(: 2.807355


In [109]:
import re

def t_Sup(scanner, token): return "Sup", token[1:]
def t_Word(scanner, token):   return "Word", token
def t_Comma(scanner, token):   return "Comma", token
def t_Dot(scanner, token):   return "Dot", token
def t_Number(scanner, token):   return "Number", token
def t_Operator(scanner, token):   return "Operator", token
def t_EqualsLine(scanner, token):   return "EqualsLine", token
def t_DoubleEquals(scanner, token):   return "DoubleEquals", token
def t_EVAL(scanner, token): return "EVAL", token[:-1]+str(eval(token[:-2]))
def t_MathBegin(scanner, token): return "MathBegin", token
def t_Newline(scanner, token): return "Newline", token
def t_Space(scanner, token): return "Space", token

def scan1(text):
    scanner1 = re.Scanner([
        (r">", t_MathBegin),
        (r"[a-zA-Z]+", t_Word),
        (r"[,:;]+", t_Comma),
        (r"[\.!]+", t_Dot),
        (r"[0-9]+", t_Number),
        (r"\n==+\n", t_EqualsLine),
        (r"==", t_DoubleEquals),
        (r"\^[0-9]+", t_Sup),
        (r"[\+*-/]", t_Operator),
        (r"\n", t_Newline),
        (r"\s+", t_Space),
        ])
    parse1 = ""
    math="false"
    calc=""
    tokens, remainder = scanner1.scan(text)
    for token in tokens:
        if math=="true":
            if token[0]=="DoubleEquals":
                math="false"
                parse1+="="+ str(eval(calc))
                calc=""
            elif token[0]=="Sup":
                calc+="**"+token[1]
                parse1+="<sup>"+token[1]+"</sup>"
            else:
                calc+=token[1]
                parse1+=token[1]
        
        elif token[0]=="MathBegin":
            math="true"
            parse1+=token[1]
        else:
            parse1+=token[1]
    print(parse1)


md="""Math in markdown
================

>3^7/9==


Then we multiply by 4 because there is 4 legs,
>3*4=="""
scan1(md)

    

Math in markdown

>3<sup>7</sup>/9=243.0


Then we multiply by 4 because there is 4 legs,
>3*4=12


In [108]:
del str

In [114]:
# python version of SQL's two-sided outer join. Naive implementation, not optimized
a = [(1,),(3,),(4,),(5,),(7,)]
b = [(1,),(2,),(4,),(5,),(8,),(9,)]

def iterate_IDs(set1, set2):
    def next_or_None(seq):
        try:
            return seq.next()
        except StopIteration:
            return None
    id1 = None
    id2 = None
    id1 = next_or_None(set1)
    id2 = next_or_None(set2)
    while id1 is not None or id2 is not None:
        if id1 is None:
            yield (None, id2)
            id2 = next_or_None(set2)
        elif id2 is None:
            yield (id1, None)
            id1 = next_or_None(set2)
        elif id1 == id2:
            yield (id1, id2)
            id1 = next_or_None(set1)
            id2 = next_or_None(set2)
        elif id1 > id2:
            yield (None, id2)
            id2 = next_or_None(set2)
        elif id2 > id1:
            yield (id1, None)
            id1 = next_or_None(set1)
    raise(StopIteration, "Both sequences run out")

import itertools as it
for id1,id2 in iterate_IDs(it.chain.from_iterable(a), it.chain.from_iterable(b)):
    print(id1, id2)



AttributeError: 'itertools.chain' object has no attribute 'next'

In [124]:
## Decorator tools

# These are some basic python tools for function management

# Decorators
# Memoization
# Call Tracing


from functools import update_wrapper

# Update wrapper
def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return update_wrapper(d(fn), fn)
    update_wrapper(_d, d)
    return _d

# Change binary function into n_ary func:

@decorator
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such that
    f(x, y, z) = f(x, f(y, z)), etc.  Also allow f(x) = x."""
    def n_ary_f(x, *args):
        return x if not args else f(x, n_ary_f(*args))
    return n_ary_f

# Example of use:
@n_ary
def seq(x, y): return ('seq', x, y)
print(seq(1), seq(1, 2), seq(1, 2, 3))

## Memoization
@decorator
def memo(f):
    """Decorator that caches the return value for each call to f(args).
    Then when called again with same args, we can just look it up."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args can't be adict key
            return f(args)
    return _f





1 ('seq', 1, 2) ('seq', 1, ('seq', 2, 3))


In [127]:
# An eventual value that automatically flattens its results when they are also eventual.

class Future:
    ### Inits a new not-yet-specified future value
    def __init__(self):
        self.__continuations = []
        
    ### Registers a continuation to run when the flattened future value is ready
    ### Returns the continuation's future result
    def continueWith(self, continuation):
        # remember to run continuation, if result not provided yet
        if self.__continuations != None:
            r = Future()
            self.__continuations.append((continuation, r))
            return r
        
        # automatic flattening
        if isinstance(self.__result, Future):
            return self.__result.continueWith(continuation)

        # run continuation inline when already finished        
        r = Future()
        v = continuation(self.__result)
        r.trySetResult(v)
        return r
    
    ### Sets this future's value, if it has not yet been set.
    ### Causes all registered continuations to be run or handed off.
    def trySetResult(self, result):
        # already set?
        cs = self.__continuations
        if cs == None: return False;
        
        # state transition
        self.__continuations = None
        self.__result = result
        
        # perform or hand-off registered continuations
        for e in cs:
            continuation, future = e
            r = self.continueWith(continuation);
            future.trySetResult(r)
        
        return True
    
    ### Determines if this future has a flattened result yet
    def hasResult(self):
        return self.__continuations == None and (not isinstance(self.__result, Future) or self.__result.hasResult())
    
    ### Gets the flattened result of this future, raising an error if it's not ready yet
    def forceGetResult(self):
        assert self.hasResult() 
        return self.__result if not isinstance(self.__result, Future) else self.__result.forceGetResult()
        
    
f1 = Future()
f2 = Future()
f3 = f1.continueWith(lambda v1: f2.continueWith(lambda v2: v1 + v2))

def out(x): print(x)
f1.continueWith(lambda x: out("f1's result = " + x))
f2.continueWith(lambda x: out("f2's result = " + x))
f3.continueWith(lambda x: out("f3's result = " + x))

f1.trySetResult(f2)
f2.trySetResult('Hello')

# outputs: 
# f2's result = Hello
# f3's result = HelloHello
# f1's result = Hello

f2's result = Hello
f3's result = HelloHello
f1's result = Hello


True

In [129]:
"""
A quick script to print out the amount of primes entered at the command line.
"""



"""
Got this from StackOverflow (http://stackoverflow.com/questions/1628949/to-find-first-n-prime-numbers-in-python). 
The regular expression use kinda blew my mind.
"""
import re, sys


def isPrime(n):
    # see http://www.noulakaz.net/weblog/2007/03/18/a-regular-expression-to-check-for-prime-numbers/
    return re.match(r'^1?$|^(11+?)\1+$', '1' * n) == None


N = int(12) # number of primes wanted (from command-line) sys.argv[1]
M = 100              # upper-bound of search space
l = list()           # result list

while len(l) < N:
    l += filter(isPrime, range(M - 100, M)) # append prime element of [M - 100, M] to l
    M += 100                                # increment upper-bound

print(l[:N]) # print result list limited to N elements

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


In [133]:
"""
Calculates amounts of rice in amount
"""
import math

def count():
    var_input = input('How many days?\n')
    x = 2
    for i in range (1,int(var_input)+1):
        print( ('Day:'+ str(i) +': ' +str(math.pow(x,i)) + ' grains of rice') )
count()

How many days?
10
Day:1: 2.0 grains of rice
Day:2: 4.0 grains of rice
Day:3: 8.0 grains of rice
Day:4: 16.0 grains of rice
Day:5: 32.0 grains of rice
Day:6: 64.0 grains of rice
Day:7: 128.0 grains of rice
Day:8: 256.0 grains of rice
Day:9: 512.0 grains of rice
Day:10: 1024.0 grains of rice


In [135]:
def vowel_remover(text):
    string = ""
    for l in text:
        if l.lower() != "a" and l.lower() != "e" and l.lower() != "i" and l.lower() != "o" and l.lower() != "u":
            string += l
    return string
print(vowel_remover("hello world!"))

hll wrld!


In [137]:
def DFS(edges, s, t, skip = []):
   if(s == t):
      return [s]
   if s not in edges: return []
   skip.append(s)
   for x in edges[s]:
       if x in skip: continue
       res = DFS(edges, x, t, skip)
       if(res):
           way = [s]
           way.extend(res)
           return way
   return []


edges = {'a':['b','c'],
         'c':['d','e'],
         'd':['f'],
         'f':['c'],
         'e':['f','g']}

way = DFS(edges, 'g', 'a')
if way:
  print(', '.join(map(str, way)) )
else:
  print('No way' )

No way


In [138]:
import random

def getlength(script):
	return sum((i['length'] for i in script))

def truncate(target_length, script):
	if getlength(script) > target_length:
		script = sorted(script, key=lambda k: (k['priority'], -k['length']))[:-1]
		return truncate(target_length, script)
	return sorted(script, key=lambda k: k['index'])

def as_text(script):
	return "\n".join([i['text'] for i in script])


priorities_and_sentences = [
(1, "...now... sitting comfortably in the chair"),
(2, "...with your feet still flat on the ground"),
(3, "...back straight and head up right"),
(2, "...make these adjustments now if you need to"),
(3, "... pause.............................."),
(1, "...your eyes ...still ...comfortably closed"),
(2, "...nice and relaxed...comfortable and relaxed..."),
(3, "... pause......................................."),
(1, "...now...I want you to notice...how heavy your head is starting to feel..."),
(1, "how heavy your head feels..."),
(3, "... pause......................................."),
(2, "really noticing the weight... of your head..."),
(3, "and how much more ...comfortable...it will feel when you let your neck relaxes ...and your head begins to fall forward ...into a much more comfortable"),
]

scriptlist = [{'priority': j[0], 'text': j[1], 'length': len(j[1]), 'index': i } for i, j in enumerate(priorities_and_sentences)]

print "500 characters"
print as_text(truncate(500, scriptlist))
print "\n300 characters"
print as_text(truncate(300, scriptlist))
print "\n200 characters"
print as_text(truncate(200, scriptlist))



SyntaxError: Missing parentheses in call to 'print'. Did you mean print("500 characters")? (<ipython-input-138-b441bdc11334>, line 34)

In [146]:
import md5
import random

sentence = "The quick brown fox jumps over the internetz"

print("start")
def hashMe(plainText):
    m = md5.md5()
    m.update(plainText)
    return m.hexdigest()

def encrypt(plainText):
    derp = "::"
    for i in plainText:
        derp += hashMe(hashMe(str(random.randint(1, 100))+i))
        derp += "::"
    return derp

def decrypt(chiper):
    plain = ""
    print("Checkpoint")
    for imp in chiper:
        print(imp)
        try:
            for b in range(1, 100):
                print("Checking", b )
                for a in range(1, 200):
                    #print "compare", hashMe(hashMe(str(b)+chr(a))), "With", i
                    if hashMe(hashMe(str(b)+chr(a))) == imp:
                        print("Debug hash check returned true:", chr(a) )
                        plain += chr(a)
        except e:
            print(e)
        return plain

        

a = encrypt(sentence)
print(a)
print("End")




ModuleNotFoundError: No module named 'md5'

In [149]:
"""
UNIT 4: Search

Your task is to maneuver a car in a crowded parking lot. This is a kind of 
puzzle, which can be represented with a diagram like this: 

| | | | | | | |  
| G G . . . Y |  
| P . . B . Y | 
| P * * B . Y @ 
| P . . B . . |  
| O . . . A A |  
| O . S S S . |  
| | | | | | | | 

A '|' represents a wall around the parking lot, a '.' represents an empty square,
and a letter or asterisk represents a car.  '@' marks a goal square.
Note that there are long (3 spot) and short (2 spot) cars.
Your task is to get the car that is represented by '**' out of the parking lot
(on to a goal square).  Cars can move only in the direction they are pointing.  
In this diagram, the cars GG, AA, SSS, and ** are pointed right-left,
so they can move any number of squares right or left, as long as they don't
bump into another car or wall.  In this diagram, GG could move 1, 2, or 3 spots
to the right; AA could move 1, 2, or 3 spots to the left, and ** cannot move 
at all. In the up-down direction, BBB can move one up or down, YYY can move 
one down, and PPP and OO cannot move.

You should solve this puzzle (and ones like it) using search.  You will be 
given an initial state like this diagram and a goal location for the ** car;
in this puzzle the goal is the '.' empty spot in the wall on the right side.
You should return a path -- an alternation of states and actions -- that leads
to a state where the car overlaps the goal.

An action is a move by one car in one direction (by any number of spaces).  
For example, here is a successor state where the AA car moves 3 to the left:

| | | | | | | |  
| G G . . . Y |  
| P . . B . Y | 
| P * * B . Y @ 
| P . . B . . |  
| O A A . . . |  
| O . . . . . |  
| | | | | | | | 

And then after BBB moves 2 down and YYY moves 3 down, we can solve the puzzle
by moving ** 4 spaces to the right:

| | | | | | | |
| G G . . . . |
| P . . . . . |
| P . . . . * *
| P . . B . Y |
| O A A B . Y |
| O . . B . Y |
| | | | | | | |

You will write the function

    solve_parking_puzzle(start, N=N)

where 'start' is the initial state of the puzzle and 'N' is the length of a side
of the square that encloses the pieces (including the walls, so N=8 here).

We will represent the grid with integer indexes. Here we see the 
non-wall index numbers (with the goal at index 31):

 |  |  |  |  |  |  |  |
 |  9 10 11 12 13 14  |
 | 17 18 19 20 21 22  |
 | 25 26 27 28 29 30 31
 | 33 34 35 36 37 38  |
 | 41 42 43 44 45 46  |
 | 49 50 51 52 53 54  |
 |  |  |  |  |  |  |  |

The wall in the upper left has index 0 and the one in the lower right has 63.
We represent a state of the problem with one big tuple of (object, locations)
pairs, where each pair is a tuple and the locations are a tuple.  Here is the
initial state for the problem above in this format:
"""

puzzle1 = (
 ('@', (31,)),
 ('*', (26, 27)), 
 ('G', (9, 10)),
 ('Y', (14, 22, 30)), 
 ('P', (17, 25, 33)), 
 ('O', (41, 49)), 
 ('B', (20, 28, 36)), 
 ('A', (45, 46)), 
 ('|', (0, 1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 23, 24, 32, 39,
        40, 47, 48, 55, 56, 57, 58, 59, 60, 61, 62, 63)))

# A solution to this puzzle is as follows:

#     path = solve_parking_puzzle(puzzle1, N=8)
#     path_actions(path) == [('A', -3), ('B', 16), ('Y', 24), ('*', 4)]

# That is, move car 'A' 3 spaces left, then 'B' 2 down, then 'Y' 3 down, 
# and finally '*' moves 4 spaces right to the goal.

# Your task is to define solve_parking_puzzle:

N = 8

def solve_parking_puzzle(start, N=N):
    """Solve the puzzle described by the starting position (a tuple 
    of (object, locations) pairs).  Return a path of [state, action, ...]
    alternating items; an action is a pair (object, distance_moved),
    such as ('B', 16) to move 'B' two squares down on the N=8 grid."""
    
# But it would also be nice to have a simpler format to describe puzzles,
# and a way to visualize states.
# You will do that by defining the following two functions:

def locs(start, n, incr=1):
    "Return a tuple of n locations, starting at start and incrementing by incr."


def grid(cars, N=N):
    """Return a tuple of (object, locations) pairs -- the format expected for
    this puzzle.  This function includes a wall pair, ('|', (0, ...)) to 
    indicate there are walls all around the NxN grid, except at the goal 
    location, which is the middle of the right-hand wall; there is a goal
    pair, like ('@', (31,)), to indicate this. The variable 'cars'  is a
    tuple of pairs like ('*', (26, 27)). The return result is a big tuple
    of the 'cars' pairs along with the walls and goal pairs."""


def show(state, N=N):
    "Print a representation of a state as an NxN grid."
    # Initialize and fill in the board.
    board = ['.'] * N**2
    for (c, squares) in state:
        for s in squares:
            board[s] = c
    # Now print it out
    for i,s in enumerate(board):
        print(s),
        if i % N == N - 1: print

# Here we see the grid and locs functions in use:

puzzle1 = grid((
    ('*', locs(26, 2)),
    ('G', locs(9, 2)),
    ('Y', locs(14, 3, N)),
    ('P', locs(17, 3, N)),
    ('O', locs(41, 2, N)),
    ('B', locs(20, 3, N)),
    ('A', locs(45, 2))))

puzzle2 = grid((
    ('*', locs(26, 2)),
    ('B', locs(20, 3, N)),
    ('P', locs(33, 3)),
    ('O', locs(41, 2, N)),
    ('Y', locs(51, 3))))

puzzle3 = grid((
    ('*', locs(25, 2)),
    ('B', locs(19, 3, N)),
    ('P', locs(36, 3)),
    ('O', locs(45, 2, N)),
    ('Y', locs(49, 3))))


# Here are the shortest_path_search and path_actions functions from the unit.
# You may use these if you want, but you don't have to.

def win_test(state):
    for item, locs in state:
        if item == '*':
            pos = locs
        if item == '@':
            goal = locs[0]
    if goal in pos:
        return True
    return False


def shortest_path_search(start, successors, is_goal):
    """Find the shortest path from start state to a state
    such that is_goal(state) is true."""
    if is_goal(start):
        return [start]
    explored = set() # set of states we have visited
    frontier = [ [start] ] # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if is_goal(state):
                    return path2
                else:
                    frontier.append(path2)
    return []

def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]

puzzle1 = (
 ('@', (31,)),
 ('*', (26, 27)), 
 ('G', (9, 10)),
 ('Y', (14, 22, 30)), 
 ('P', (17, 25, 33)), 
 ('O', (41, 49)), 
 ('B', (20, 28, 36)), 
 ('A', (45, 46)), 
 ('|', (0, 1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 23, 24, 32, 39,
        40, 47, 48, 55, 56, 57, 58, 59, 60, 61, 62, 63)))

print(win_test(puzzle1) )
show(puzzle1, 8)

False
|
|
|
|
|
|
|
|
|
G
G
.
.
.
Y
|
|
P
.
.
B
.
Y
|
|
P
*
*
B
.
Y
@
|
P
.
.
B
.
.
|
|
O
.
.
.
A
A
|
|
O
.
.
.
.
.
|
|
|
|
|
|
|
|
|


In [152]:
"""
Showing how to use star arguments. Word word word word.
"""

def function(one, two, three):
    print('Function called with:', one, two, three )
    
# This will work just fine
args = [1, 2, 3]
function(*args)

# This will work just fine, as well
args = '456'
function(*args)

# Call a function with any number of arguments given
def call(f, args):
    # Unpack the list of args and give to the function
    # The cool thing is that we don't have to know how many arguments
    # the function takes, we just use the * operator and it unpacks them for us.
    f(*args)
    

# Pass in our function and arguments
args = [7, 8, 9]
call(function, args)

Function called with: 1 2 3
Function called with: 4 5 6
Function called with: 7 8 9


In [156]:
"""
testing exclude regex with amazon
"""

import re


good_urls = (
    'http://www.amazon.com/exec/obidos/ASIN/B004NVUXH2/gasmencommon',
    'http://www.amazon.com/exec/obidos/ASIN/B004NVUXH2/gasmencommon-10',
    'http://www.amazon.com/exec/obidos/ASIN/B004NVUXH2/gasmencommon-2a',
)

bad_urls = (
    'http://www.amazon.com/exec/obidos/ASIN/B004NVUXH2/gasmencommon-20',
    'http://www.amazon.com/exec/obidos/ASIN/B004NVUXH2/gasmencommon-20-sdsdf',
    'http://www.amazon.com/exec/obidos/ASIN/B004N-20VUXH2/gasmencommon',
)


#reg = re.compile(r'^.*(gp/product|dp|ASIN)/(B[A-Za-z0-9]+)((?!\-2\d).)*$')
#reg = re.compile(r'^.*(gp\/product|dp|ASIN)/(?P<ext_id>B[A-Za-z0-9]+)((?!\-2\d).)*$')
reg = re.compile(r'^.*(gp\/product|dp|ASIN)\/(?P<ext_id>B[A-Za-z0-9]+)(?!.*\-(20|21).*$)')

def test_urls():
    for url in good_urls:
        match = reg.search(url)
        assert match
        print(match.groups()[1] )
        
    for url in bad_urls:
        assert not reg.search(url)


if __name__ == '__main__':
    test_urls()

B004NVUXH2
B004NVUXH2
B004NVUXH2


In [161]:
"""
Playing with collections and generators
"""

from array import *
q = [1,2,3,4,5]
t = [s for s in q if s > 3]
w = map(lambda q: q,filter(lambda d:d>2, q))
#print t, w

p = []
for i in range(5):
    p.append([x for x in range(6)])
    
#print p

def add_letters(l):
    for y in l:
        yield y

hash = {'help' : ['h','e','l','p']}

h = add_letters(hash['help'])
print(''.join(hash['help']))
print(''.join([d for d in h]))



class Mega(object):

    def __init__(self, name):
        print('Setting up the Mega object')
        self.name = name
    def __str__(self):
        return self.name
        
m = Mega('Arkadiusz')
print(str(m))

def cheat_args(function):
    def wrapper(*args, **kwargs):
        if len(args[0]) > 0:
            args[0][0] = 'It\'s sneaky'
        return function(*args, **kwargs)
    return wrapper
    
@cheat_args    
def writy(to_write):
    print(to_write)
    for i in to_write:
        print(i)
        
#writy(['One', 'Two', 'Three'])

life = [[0 for i in range(4)] for i in range(4)]
print(life [1][1])


    




help
help
Setting up the Mega object
Arkadiusz
0


In [170]:
"""
Compute basic color stuff like gradients and intermediate colors.
"""
def arrayMultiply(array, c):
    return [element*c for element in array]

def arraySum(a, b):
    return map(sum, zip(a,b))

def intermediate(a, b, ratio):
    aComponent = arrayMultiply(a, ratio)
    bComponent = arrayMultiply(b, 1-ratio)
    return arraySum(aComponent, bComponent)

def gradient(a, b, steps):
    steps = [n/float(steps) for n in range(steps)]
    for step in steps:
        print(intermediate(a, b, step))

#print arrayMultiply((1,2,3), 0.3)
#print arraySum((1,2,3), (0.5, 0.5, 0.5))
#print intermediate((1,2,3), (3,2,1), 0.5)
#print gradient(None, None, 5)


pureBlue = (8, 123, 157)
pureYellowGreen = (0,84,166)

gradient(pureBlue, pureYellowGreen, 6)




In [166]:
# ruman numeral puzzle from nytimes 12-31-12
from random import randint

vals={"1":1,"5":2,"10":2,"50":2,"1000":4}

sticks=10
iterations=1000
print(vals)
print(len(vals))
for iteration in range(iterations):
    currentSticks=sticks
    while currentSticks>0:
        index=randint(0,len(vals)-1)
        keys=vals.keys()
        val=vals[keys[index]]
        

{'1': 1, '5': 2, '10': 2, '50': 2, '1000': 4}
5


TypeError: 'dict_keys' object is not subscriptable

In [173]:
"""
Simple Program Compare 2 csv list
"""
List1=[1, 2, 3, 121, 4, 5, 6, 7, 8, 9, 10]
List2=[1, 2, 3, 4, 4, 6, 7, 8, 11, 77]
List3=list(set(List1)-set(List2))
print(List3 )

[9, 10, 5, 121]


In [175]:
"""
Project Euler #15

Recursive counting of Lattice Path of NxM Grid

Note: Doesn't work well with high numbers (i.e. 20)

LatticePath_v2 uses binomial coefficient of the NxN grid to solve for the number of LatticePath  
"""
import math

def LatticePath(down, right):
    
    if(down == 0 or right == 0):
        return 1
    
    else:
        return LatticePath(down -1, right) + LatticePath(down, right -1)
    
    
def LatticePath_v2(down, right):
	numPaths = 0
    
	return math.factorial(down + right) / (math.factorial(down)**2)
    
	
print LatticePath(3,3)


print LatticePath_v2(20,20)

SyntaxError: invalid syntax (<ipython-input-175-5a0b3a7cc177>, line 27)

In [176]:
class mylist():
    def __init__(self,l):
        self.l = l
    def extend(self,index):
        diff = index + 1 - len(self.l)
        if diff > 0:
            self.l.extend([ None for x in range(diff) ] )
    def __getitem__(self,index):
        # self.extend(index) ## depending on your taste
        return self.l[index]
    def __setitem__(self,index,value):
        self.extend(index) 
        self.l[index] = value
    def __repr__(self):
        return self.l.__repr__()
    
a = mylist([0,1,2,3])
print 'List: %s' % a
# print a[5]  ### this will throw an 'list index out of range' unless 
              ### you comment out the bit in __getitem__
a[5] = 5
print 'List: %s' % a
print a[5]




SyntaxError: Missing parentheses in call to 'print'. Did you mean print('List: %s' % a)? (<ipython-input-176-35ff0cb67999>, line 18)

In [177]:
from functools import reduce

# constants used in the multGF2 function
mask1 = mask2 = polyred = None

def setGF2(degree, irPoly):
    """Define parameters of binary finite field GF(2^m)/g(x)
       - degree: extension degree of binary field
       - irPoly: coefficients of irreducible polynomial g(x)
    """
    def i2P(sInt):
        """Convert an integer into a polynomial"""
        return [(sInt >> i) & 1
                for i in reversed(range(sInt.bit_length()))]    
    
    global mask1, mask2, polyred
    mask1 = mask2 = 1 << degree
    mask2 -= 1
    polyred = reduce(lambda x, y: (x << 1) + y, i2P(irPoly)[1:])
        
def multGF2(p1, p2):
    """Multiply two polynomials in GF(2^m)/g(x)"""
    p = 0
    while p2:
        if p2 & 1:
            p ^= p1
        p1 <<= 1
        if p1 & mask1:
            p1 ^= polyred
        p2 >>= 1
    return p & mask2

if __name__ == "__main__":
  
    # Define binary field GF(2^3)/x^3 + x + 1
    setGF2(3, 0b1011)

    # Evaluate the product (x^2 + x + 1)(x^2 + 1)
    print("{:02x}".format(multGF2(0b111, 0b101)))
    
    # Define binary field GF(2^8)/x^8 + x^4 + x^3 + x + 1
    # (used in the Advanced Encryption Standard-AES)
    setGF2(8, 0b100011011)
    
    # Evaluate the product (x^7)(x^7 + x + 1)
    print("{:02x}".format(multGF2(0b10000000, 0b10000011)))

06
01


In [172]:
# http://pythonfiddle.com/artemis-master-encryption-algorithm/